# LSTM Stock Prediction Notebook

## Table of Contents
1. [Introduction](#1-introduction)
2. [Libraries](#2-libraries)  
3. [Data Collection](#3-data-collection)  
4. [Data Exploration](#4-data-exploration)  
5. [Feature Engineering](#5-feature-engineering)  
6. [Model Architecture](#6-model-architecture)  
7. [Training & Evaluation](#7-training--evaluation)  
8. [Results & Diagnostics](#8-results--diagnostics)  
9. [Conclusion](#9-conclusion)

## 1. Introduction

## 2. Libraries

In [3]:
# Core Libraries 
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Machine Learning Libraries
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import mplfinance as mpf

# Alpaca Libraries
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame